In [1]:
import pandas as pd

In [7]:
msoa_ward_lookup = pd.read_csv(
    "https://opendata.arcgis.com/datasets/fcb3d6b3dc834e3ca3b38756b8b023f2_0.csv",
    index_col='MSOA11CD'
)

In [8]:
msoa_ward_lookup

,MSOA11NM,WD17CD,WD17NM,WD17NMW,LAD17CD,LAD17NM,FID
MSOA11CD,,,,,,,
E02000001,City of London 001,E05009288,Aldersgate,NaN,E09000001,City of London,1
E02000002,Barking and Dagenham 001,E05000029,Chadwell Heath,NaN,E09000002,Barking and Dagenham,2
E02000003,Barking and Dagenham 002,E05000029,Chadwell Heath,NaN,E09000002,Barking and Dagenham,3
E02000004,Barking and Dagenham 003,E05000030,Eastbrook,NaN,E09000002,Barking and Dagenham,4
E02000005,Barking and Dagenham 004,E05000040,Valence,NaN,E09000002,Barking and Dagenham,5
...,...,...,...,...,...,...,...
W02000320,Blaenau Gwent 007,W05000940,Cwm,Cwm,W06000019,Blaenau Gwent,7197
W02000113,Powys 017,W05000346,St. Mary,St. Mary,W06000023,Powys,7198
W02000321,Blaenau Gwent 008,W05000941,Cwmtillery,Cwmtyleri,W06000019,Blaenau Gwent,7199


In [9]:
brc_df = pd.read_csv(
    "https://raw.githubusercontent.com/britishredcrosssociety/covid-19-vulnerability/master/output/vulnerability-MSOA-UK.csv",
    index_col='Code'
)

In [12]:
brc_df = brc_df.join(msoa_ward_lookup)
brc_df.to_csv("data/brc_vulnerability_msoa.csv")

In [21]:
brc_df

,Name,Clinical Vulnerability score,Clinical Vulnerability rank,Clinical Vulnerability decile,Health/Wellbeing Vulnerability score,Health/Wellbeing Vulnerability rank,Health/Wellbeing Vulnerability decile,Economic Vulnerability score,Economic Vulnerability rank,Economic Vulnerability decile,...,Vulnerability score,Vulnerability rank,Vulnerability decile,MSOA11NM,WD17CD,WD17NM,WD17NMW,LAD17CD,LAD17NM,FID
Code,,,,,,,,,,,,,,,,,,,,,
E02000001,City of London,38.773821,181,1,61.517572,762,2,13.917755,138.0,1.0,...,56.994782,1977,3,City of London 001,E05009288,Aldersgate,NaN,E09000001,City of London,1.0
E02000002,Marks Gate,125.401469,1112,2,55.974971,577,1,211.181991,5085.0,8.0,...,55.325106,1866,3,Barking and Dagenham 001,E05000029,Chadwell Heath,NaN,E09000002,Barking and Dagenham,2.0
E02000003,Chadwell Heath East,135.324882,1282,2,41.259355,223,1,104.132336,2968.0,5.0,...,45.744399,1273,2,Barking and Dagenham 002,E05000029,Chadwell Heath,NaN,E09000002,Barking and Dagenham,3.0
E02000004,Eastbrookend,114.455532,963,2,149.294578,4642,7,164.217818,4365.0,7.0,...,72.960540,2992,5,Barking and Dagenham 003,E05000030,Eastbrook,NaN,E09000002,Barking and Dagenham,4.0
E02000005,Becontree Heath,170.639457,1775,3,49.309085,375,1,154.779439,4174.0,7.0,...,53.134819,1738,3,Barking and Dagenham 004,E05000040,Valence,NaN,E09000002,Barking and Dagenham,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95ZZ14S1,South_1,65.777705,566,7,63.054563,689,8,NaN,NaN,NaN,...,63.952904,518,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95ZZ14S2,South_2,31.811341,273,4,43.059359,500,6,NaN,NaN,NaN,...,31.094235,161,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95ZZ15W1,Victoria Bridge,43.561662,381,5,30.548408,342,4,NaN,NaN,NaN,...,57.742850,460,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
brc_df.columns

Index(['Name', 'Clinical Vulnerability score', 'Clinical Vulnerability rank',
       'Clinical Vulnerability decile', 'Health/Wellbeing Vulnerability score',
       'Health/Wellbeing Vulnerability rank',
       'Health/Wellbeing Vulnerability decile', 'Economic Vulnerability score',
       'Economic Vulnerability rank', 'Economic Vulnerability decile',
       'Social Vulnerability score', 'Social Vulnerability rank',
       'Social Vulnerability decile', 'Vulnerability score',
       'Vulnerability rank', 'Vulnerability decile', 'MSOA11NM', 'WD17CD',
       'WD17NM', 'WD17NMW', 'LAD17CD', 'LAD17NM', 'FID'],
      dtype='object')

In [24]:
components = [
    'Clinical Vulnerability',
    'Health/Wellbeing Vulnerability',
    'Economic Vulnerability',
    'Social Vulnerability',
    'Vulnerability',
]

brc_df_ward = brc_df.groupby('WD17CD').agg({
    **{f"{c} score": 'mean' for c in components},
    'WD17NM': 'first',
    'WD17NMW': 'first',
    'LAD17CD': 'first',
    'LAD17NM': 'first',
})
for c in components:
    brc_df_ward.loc[:, f"{c} rank"] = brc_df_ward[f"{c} score"].rank()
    brc_df_ward.loc[:, f"{c} decile"] = pd.qcut(brc_df_ward[f"{c} score"], 10, labels=False)
    brc_df_ward.loc[:, f"{c} quintile"] = pd.qcut(brc_df_ward[f"{c} score"], 5, labels=False)
brc_df_ward.to_csv("data/brc_vulnerability_ward_2017.csv")
brc_df_ward

,Clinical Vulnerability score,Health/Wellbeing Vulnerability score,Economic Vulnerability score,Social Vulnerability score,Vulnerability score,WD17NM,WD17NMW,LAD17CD,LAD17NM,Clinical Vulnerability rank,...,Health/Wellbeing Vulnerability quintile,Economic Vulnerability rank,Economic Vulnerability decile,Economic Vulnerability quintile,Social Vulnerability rank,Social Vulnerability decile,Social Vulnerability quintile,Vulnerability rank,Vulnerability decile,Vulnerability quintile
WD17CD,,,,,,,,,,,,,,,,,,,,,
E05000026,80.638865,15.592908,115.540490,318.636755,64.869394,Abbey,NaN,E09000002,Barking and Dagenham,587.0,...,0,2792.0,4,2,4874.0,8,4,1959.0,3,1
E05000027,135.168761,38.165078,197.786175,226.822670,56.961133,Alibon,NaN,E09000002,Barking and Dagenham,1183.0,...,0,4174.0,7,3,3512.0,6,3,1576.0,2,1
E05000028,126.813906,42.364313,185.613783,278.509790,69.972516,Becontree,NaN,E09000002,Barking and Dagenham,1060.0,...,0,4021.0,7,3,4523.0,8,4,2216.0,3,1
E05000029,130.363176,48.617163,157.657164,225.901929,50.534752,Chadwell Heath,NaN,E09000002,Barking and Dagenham,1112.0,...,0,3617.0,6,3,3487.0,6,3,1214.0,2,1
E05000030,123.204194,116.658630,132.011640,229.369469,61.820606,Eastbrook,NaN,E09000002,Barking and Dagenham,1009.0,...,2,3140.0,5,2,3587.0,6,3,1805.0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W05001029,126.799440,80.448318,289.933865,209.598714,98.888795,Bryn and Cwmavon,Bryn a Chwmafan,W06000012,Neath Port Talbot,1059.0,...,1,4969.0,8,4,3016.0,5,2,3648.0,6,3
W05001032,153.044054,120.455634,190.285817,223.991361,120.823337,Port Talbot,Port Talbot,W06000012,Neath Port Talbot,1372.0,...,2,4084.0,7,3,3435.0,6,3,4407.0,7,3
W05001033,203.222208,94.802866,507.914624,201.517949,171.258727,Sandfields East,Sandfields - Dwyrain,W06000012,Neath Port Talbot,1951.0,...,1,5599.0,9,4,2746.0,4,2,5321.0,9,4


In [25]:

brc_df_la = brc_df.groupby('LAD17CD').agg({
    **{f"{c} score": 'mean' for c in components},
    'LAD17NM': 'first',
})
for c in components:
    brc_df_la.loc[:, f"{c} rank"] = brc_df_la[f"{c} score"].rank()
    brc_df_la.loc[:, f"{c} decile"] = pd.qcut(brc_df_la[f"{c} score"], 10, labels=False)
    brc_df_la.loc[:, f"{c} quintile"] = pd.qcut(brc_df_la[f"{c} score"], 5, labels=False)
brc_df_la.to_csv("data/brc_vulnerability_la.csv")
brc_df_la

,Clinical Vulnerability score,Health/Wellbeing Vulnerability score,Economic Vulnerability score,Social Vulnerability score,Vulnerability score,LAD17NM,Clinical Vulnerability rank,Clinical Vulnerability decile,Clinical Vulnerability quintile,Health/Wellbeing Vulnerability rank,...,Health/Wellbeing Vulnerability quintile,Economic Vulnerability rank,Economic Vulnerability decile,Economic Vulnerability quintile,Social Vulnerability rank,Social Vulnerability decile,Social Vulnerability quintile,Vulnerability rank,Vulnerability decile,Vulnerability quintile
LAD17CD,,,,,,,,,,,,,,,,,,,,,
E06000001,436.250822,144.955346,393.597204,112.388173,126.604049,Hartlepool,268.0,7,3,233.0,...,3,347.0,9,4,3.0,0,0,292.0,8,4
E06000002,295.547328,169.323377,320.970277,108.496546,104.217688,Middlesbrough,175.0,5,2,293.0,...,4,343.0,9,4,2.0,0,0,238.0,6,3
E06000003,480.954260,172.805584,276.166697,139.633835,123.744696,Redcar and Cleveland,296.0,8,4,298.0,...,4,327.0,9,4,15.0,0,0,284.0,8,4
E06000004,383.980007,142.471861,233.603553,131.576885,94.519353,Stockton-on-Tees,231.0,6,3,226.0,...,3,301.0,8,4,7.0,0,0,208.0,5,2
E06000005,482.697597,191.690089,202.446973,131.765894,118.178236,Darlington,297.0,8,4,324.0,...,4,275.0,7,3,8.0,0,0,276.0,7,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
W06000020,93.268090,115.838415,245.892700,171.422059,100.910613,Torfaen,31.0,0,0,137.0,...,1,307.0,8,4,62.0,1,0,229.0,6,3
W06000021,38.758204,94.091178,72.910944,296.963085,69.497063,Monmouthshire,3.0,0,0,56.0,...,0,59.0,1,0,299.0,8,4,115.0,3,1
W06000022,101.386528,114.475925,197.158436,206.406921,104.695274,Newport,39.0,1,0,130.0,...,1,271.0,7,3,153.0,4,2,239.0,6,3
